In [67]:
import time
import random
import hashlib
import os
import binascii
import string

from cryptography.hazmat.primitives import asymmetric, hashes, kdf, ciphers, padding
from cryptography.hazmat.backends import default_backend

print("Librerías necesarias importadas.")

def generate_random_text_file(num_words, filename):
    def _generate_word():
        length = random.randint(3, 10)
        return ''.join(random.choice(string.ascii_lowercase) for _ in range(length))

    words = [_generate_word() for _ in range(num_words)]
    with open(filename, 'w') as f:
        f.write(' '.join(words))
    print(f"Se generaron {num_words} palabras aleatorias y se guardaron en '{filename}'.")

print("Función auxiliar 'generate_random_text_file' definida.")

Librerías necesarias importadas.
Función auxiliar 'generate_random_text_file' definida.


In [68]:
word_counts = [10, 100, 1000, 10000, 100000, 1000000, 10000000]
file_paths = {}

print("Verificando y generando archivos de texto para todos los tamaños...")
for num_words in word_counts:
    filename = f"input_shared_{num_words}.txt"
    file_paths[num_words] = filename

    if not os.path.exists(filename):
        generate_random_text_file(num_words, filename)
    else:
        print(f"El archivo '{filename}' ya existe. Se utilizará el archivo existente.")

print("Archivos de texto listos para usar.")

Verificando y generando archivos de texto para todos los tamaños...
El archivo 'input_shared_10.txt' ya existe. Se utilizará el archivo existente.
El archivo 'input_shared_100.txt' ya existe. Se utilizará el archivo existente.
El archivo 'input_shared_1000.txt' ya existe. Se utilizará el archivo existente.
El archivo 'input_shared_10000.txt' ya existe. Se utilizará el archivo existente.
El archivo 'input_shared_100000.txt' ya existe. Se utilizará el archivo existente.
El archivo 'input_shared_1000000.txt' ya existe. Se utilizará el archivo existente.
El archivo 'input_shared_10000000.txt' ya existe. Se utilizará el archivo existente.
Archivos de texto listos para usar.


## Implementación y Medición RC4

In [75]:
import pandas as pd
from cryptography.hazmat.primitives.ciphers import Cipher
from cryptography.hazmat.decrepit.ciphers.algorithms import ARC4
from cryptography.hazmat.backends import default_backend

rc4_results = []

print("Iniciando la medición de rendimiento de RC4...")

for num_words in word_counts:
    filename = file_paths[num_words]

    start_time = time.time()
    with open(filename, 'r') as f:
        plaintext_str = f.read()
    plaintext = plaintext_str.encode('utf-8')
    T_E1 = time.time() - start_time

    start_time = time.time()
    key = os.urandom(16)
    print(f"  Clave de cifrado/descifrado (RC4): {key.hex()}")
    T_E2 = time.time() - start_time

    start_time = time.time()
    cipher = Cipher(ARC4(key), mode=None, backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()
    T_E3 = time.time() - start_time

    start_time = time.time()
    decryptor = cipher.decryptor()
    decrypted_text_bytes = decryptor.update(ciphertext) + decryptor.finalize()
    T_E4 = time.time() - start_time

    T_Total = T_E1 + T_E2 + T_E3 + T_E4

    input_chars = len(plaintext_str)
    output_chars = len(ciphertext)

    rc4_results.append({
        "algoritmo": "RC4",
        "#palabras": num_words,
        "#caracteres_entrada": input_chars,
        "#caracteres_salida": output_chars,
        "T-E1": T_E1,
        "T-E2": T_E2,
        "T-E3": T_E3,
        "T-E4": T_E4,
        "T_Total": T_Total
    })

    print(f"Procesamiento completado para {num_words} palabras.")

print("Resultados de rendimiento de RC4:")
print(rc4_results)

print("Medición de rendimiento de RC4 completada.")

Iniciando la medición de rendimiento de RC4...
  Clave de cifrado/descifrado (RC4): c9eac2419d890060f956330989e0e0e9
Procesamiento completado para 10 palabras.
  Clave de cifrado/descifrado (RC4): 2dceb31151d94cabdbf2454ab62c4ea7
Procesamiento completado para 100 palabras.
  Clave de cifrado/descifrado (RC4): 251100035079e62bdcac774ef9e872f7
Procesamiento completado para 1000 palabras.
  Clave de cifrado/descifrado (RC4): 718d4c70ef1559e4d893b2db1812051c
Procesamiento completado para 10000 palabras.
  Clave de cifrado/descifrado (RC4): 8d1ecb5a03826b77e6174d1f3af4fd7c
Procesamiento completado para 100000 palabras.
  Clave de cifrado/descifrado (RC4): 993c1b600924715188d404f12fa87f29
Procesamiento completado para 1000000 palabras.
  Clave de cifrado/descifrado (RC4): a9a9c2fb2663c7ed7f35d7f32365b55c
Procesamiento completado para 10000000 palabras.
Resultados de rendimiento de RC4:
[{'algoritmo': 'RC4', '#palabras': 10, '#caracteres_entrada': 70, '#caracteres_salida': 70, 'T-E1': 0.00014

## Implementación y Medición Diffie-Hellman con Cifrado Simétrico


In [76]:
import pandas as pd
import time
import os

from cryptography.hazmat.primitives import ciphers, hashes, padding
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.backends import default_backend

dh_aes_results = []

print("Iniciando la medición de rendimiento de Diffie-Hellman con AES...")

for num_words in word_counts:
    filename = file_paths[num_words]

    start_time = time.time()
    with open(filename, 'r') as f:
        plaintext_str = f.read()
    plaintext = plaintext_str.encode('utf-8')
    T_E1 = time.time() - start_time

    start_time = time.time()

    parameters = dh.generate_parameters(generator=2, key_size=1024, backend=default_backend())

    alice_private_key = parameters.generate_private_key()
    alice_public_key = alice_private_key.public_key()

    bob_private_key = parameters.generate_private_key()
    bob_public_key = bob_private_key.public_key()

    print(f"  Clave pública de Alice (DH): {hex(alice_public_key.public_numbers().y)}")
    print(f"  Clave pública de Bob (DH): {hex(bob_public_key.public_numbers().y)}")

    alice_shared_key = alice_private_key.exchange(bob_public_key)

    salt = os.urandom(16)
    info = b"handshake data"

    kdf_obj = HKDF(algorithm=hashes.SHA256(), length=32, salt=salt, info=info, backend=default_backend())
    aes_key = kdf_obj.derive(alice_shared_key)

    iv = os.urandom(16)

    print(f"  Clave AES derivada (DH): {aes_key.hex()}")

    T_E2 = time.time() - start_time

    start_time = time.time()
    cipher = ciphers.Cipher(ciphers.algorithms.AES(aes_key), ciphers.modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    padder = padding.PKCS7(ciphers.algorithms.AES.block_size).padder()
    padded_plaintext = padder.update(plaintext) + padder.finalize()
    ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()
    T_E3 = time.time() - start_time

    start_time = time.time()
    decryptor = cipher.decryptor()
    unpadder = padding.PKCS7(ciphers.algorithms.AES.block_size).unpadder()
    decrypted_padded_text = decryptor.update(ciphertext) + decryptor.finalize()
    decrypted_text_bytes = unpadder.update(decrypted_padded_text) + unpadder.finalize()

    T_E4 = time.time() - start_time

    T_Total = T_E1 + T_E2 + T_E3 + T_E4

    input_chars = len(plaintext_str)
    output_chars = len(ciphertext)

    dh_aes_results.append({
        "algoritmo": "DH_AES",
        "#palabras": num_words,
        "#caracteres_entrada": input_chars,
        "#caracteres_salida": output_chars,
        "T-E1": T_E1,
        "T-E2": T_E2,
        "T-E3": T_E3,
        "T-E4": T_E4,
        "T_Total": T_Total
    })

    print(f"Procesamiento completado para {num_words} palabras.")

print("Resultados de rendimiento de Diffie-Hellman con AES:")
print(dh_aes_results)
print("Medición de rendimiento de Diffie-Hellman con AES completada.")

Iniciando la medición de rendimiento de Diffie-Hellman con AES...
  Clave pública de Alice (DH): 0x1f4b7d2fb3d525c2c2eb8705aeded3c246f7caa7b7d6f940aebe4c163e7d6d0cd2a1cdb619efed48aa424123e91530d20f98e2eef255981083ff68e29a3e922a10a2f5612afd7e5fb144919435fbefae07688831680e7bbe19b56788ae4be685a156e308f1524be12eb72aa9e9fc7e897ab68c54b513d92b0c3ccb792a9cb571
  Clave pública de Bob (DH): 0x130465d746b115127b968453f099ab814c13128700922f0fd3d793ffbc4d3a54c2f221fb765b9331dc6ebb092b38252e6a35d5a6979d354ac053698bbd84fd9169876908993eab7e80480f894bc125b614d609ceb50011bd6790684697e288c1633138eda00baf50567eb5cc2ec575ef1c519e55cdd137989ebfe4dd49911195
  Clave AES derivada (DH): 485651e9780c775e3ed15cd2fadc06a3e84e1c8ef5a0e8bb18ddad5363e36a87
Procesamiento completado para 10 palabras.
  Clave pública de Alice (DH): 0x162bea49e701c3c9d4499e5360e0eedd4dc916979af9e0eacab99b0e6e7ce6fe865d9edd3960728c14dc435720c7c969252e77aba9cf3993251e2a268148bc704c9681a40b86c525377948828c2012af0c14c2b8ffe1649d7d3b513792a4

## Implementación y Medición SHA-512


In [77]:
import time
import os
import hashlib

sha512_results = []

print("Iniciando la medición de rendimiento de SHA-512...")

for num_words in word_counts:
    filename = file_paths[num_words]

    start_time = time.time()
    with open(filename, 'r') as f:
        plaintext_str = f.read()
    plaintext_bytes = plaintext_str.encode('utf-8')
    T_E1 = time.time() - start_time

    start_time = time.time()
    hasher = hashlib.sha512()
    hasher.update(plaintext_bytes)
    hashed_data = hasher.hexdigest()
    print(f"  Hash SHA-512 generado: {hashed_data}")
    T_E_Hashing = time.time() - start_time

    T_Total = T_E1 + T_E_Hashing

    input_chars = len(plaintext_str)
    output_chars = len(hashed_data)

    sha512_results.append({
        "algoritmo": "SHA-512",
        "#palabras": num_words,
        "#caracteres_entrada": input_chars,
        "#caracteres_salida": output_chars,
        "T-E1": T_E1,
        "T-E2": 0,
        "T-E3": T_E_Hashing,
        "T-E4": 0,
        "T_Total": T_Total
    })

    print(f"Procesamiento completado para {num_words} palabras.")

print("Resultados de rendimiento de SHA-512:")
print(sha512_results)

print("Medición de rendimiento de SHA-512 completada.")

Iniciando la medición de rendimiento de SHA-512...
  Hash SHA-512 generado: c76ab6245d065beffed35afe90ea6974001d67b9aafc4bb1830c5216a057d9446268ecab9611ba549196b54542fd320d7d604ba9585d0e27322794826509d81a
Procesamiento completado para 10 palabras.
  Hash SHA-512 generado: d0929da3100407ccb52436599fcb0f715954c5f2857b99577cc3446c0e745decf0fde0c8c68995ca107f2cd6ca8814a966185f9935890e0a97340de224f0c6b0
Procesamiento completado para 100 palabras.
  Hash SHA-512 generado: 219c6acba9196a2741ebec397a65900d1eb78b245eb171a1338c3cca71cf4fbccf5525d0ff8d08a644c674479d99d071498ddd19e88146a6c3e49746db4d1be1
Procesamiento completado para 1000 palabras.
  Hash SHA-512 generado: 830e01b73255e3cc05a2dd3cb30e5f87907b1d87cb62debe048fb8898179a309b02fd641b36e8b57904a9d45f0f474d0780487d95901df5178abf590b24f4f9e
Procesamiento completado para 10000 palabras.
  Hash SHA-512 generado: 2e3d165a9f1281d4fcf121b3376b684ba132906d1982b236b91a0e00c8e09dbc82ac8ac7245a554a2fdf23cf118a788ba726ca9d78b7ad06127573df5885ceb8
P

## Consolidación de Resultados

### Subtask:
Recopilar todos los datos de tiempo y tamaño de los pasos anteriores para cada algoritmo y tamaño de archivo en una estructura de datos adecuada para su posterior visualización.


**Reasoning**:
The subtask requires consolidating the performance metrics from RC4, Diffie-Hellman with AES, and SHA-512 into a single list. I will create an empty list and then extend it with the results from each algorithm, finally printing the consolidated list.



In [78]:
all_results = []
all_results.extend(rc4_results)
all_results.extend(dh_aes_results)
all_results.extend(sha512_results)

print("Resultados consolidados (all_results):")
print(all_results)
print(f"Total de entradas en all_results: {len(all_results)}")

Resultados consolidados (all_results):
[{'algoritmo': 'RC4', '#palabras': 10, '#caracteres_entrada': 70, '#caracteres_salida': 70, 'T-E1': 0.00014638900756835938, 'T-E2': 3.075599670410156e-05, 'T-E3': 0.0001761913299560547, 'T-E4': 3.0517578125e-05, 'T_Total': 0.0003838539123535156}, {'algoritmo': 'RC4', '#palabras': 100, '#caracteres_entrada': 754, '#caracteres_salida': 754, 'T-E1': 0.0024330615997314453, 'T-E2': 2.6702880859375e-05, 'T-E3': 0.00011587142944335938, 'T-E4': 2.6464462280273438e-05, 'T_Total': 0.002602100372314453}, {'algoritmo': 'RC4', '#palabras': 1000, '#caracteres_entrada': 7512, '#caracteres_salida': 7512, 'T-E1': 0.00010466575622558594, 'T-E2': 1.9311904907226562e-05, 'T-E3': 8.797645568847656e-05, 'T-E4': 4.0531158447265625e-05, 'T_Total': 0.0002524852752685547}, {'algoritmo': 'RC4', '#palabras': 10000, '#caracteres_entrada': 74568, '#caracteres_salida': 74568, 'T-E1': 0.00013256072998046875, 'T-E2': 1.6450881958007812e-05, 'T-E3': 0.00031638145446777344, 'T-E4':

## Generación de Tabla Resumen

### Subtask:
Create a summary table (Pandas DataFrame) with the consolidated results.


**Reasoning**:
The subtask requires creating a pandas DataFrame from the consolidated results. I will import pandas, convert the `all_results` list into a DataFrame, and then display its head to verify the structure as instructed.



In [79]:
import pandas as pd

summary_df = pd.DataFrame(all_results)

print("DataFrame resumen creado. Primeras 5 filas:")
print(summary_df.head())

print(f"Total de entradas en summary_df: {len(summary_df)}")

DataFrame resumen creado. Primeras 5 filas:
  algoritmo  #palabras  #caracteres_entrada  #caracteres_salida      T-E1  \
0       RC4         10                   70                  70  0.000146   
1       RC4        100                  754                 754  0.002433   
2       RC4       1000                 7512                7512  0.000105   
3       RC4      10000                74568               74568  0.000133   
4       RC4     100000               748950              748950  0.001658   

       T-E2      T-E3      T-E4   T_Total  
0  0.000031  0.000176  0.000031  0.000384  
1  0.000027  0.000116  0.000026  0.002602  
2  0.000019  0.000088  0.000041  0.000252  
3  0.000016  0.000316  0.000231  0.000697  
4  0.000028  0.002867  0.002065  0.006618  
Total de entradas en summary_df: 21


## Tarea Final

### Subtask:
Present the summary table with all obtained results and provide a brief explanation of the observations on the performance of each algorithm with different input sizes.


In [80]:
print("Tabla resumen final con todos los resultados:")
display(summary_df)

Tabla resumen final con todos los resultados:


,algoritmo,#palabras,#caracteres_entrada,#caracteres_salida,T-E1,T-E2,T-E3,T-E4,T_Total
0,RC4,10,70,70,0.000146,0.000031,0.000176,0.000031,0.000384
1,RC4,100,754,754,0.002433,0.000027,0.000116,0.000026,0.002602
2,RC4,1000,7512,7512,0.000105,0.000019,0.000088,0.000041,0.000252
3,RC4,10000,74568,74568,0.000133,0.000016,0.000316,0.000231,0.000697
4,RC4,100000,748950,748950,0.001658,0.000028,0.002867,0.002065,0.006618
5,RC4,1000000,7499596,7499596,0.012596,0.000049,0.032541,0.027081,0.072267
6,RC4,10000000,74997531,74997531,0.272594,0.000506,0.335145,0.340603,0.948848
7,DH_AES,10,70,80,0.012484,1.243010,0.006395,0.004464,1.266353
8,DH_AES,100,754,768,0.000118,0.504534,0.000108,0.000056,0.504816
9,DH_AES,1000,7512,7520,0.000093,3.772004,0.000116,0.000037,3.772250
